In [1]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

## Absolute MMR Assignment Problem

Basado en Jordi Pereira, Igor Averbakh, Exact and heuristic algorithms for the interval data robust assignment problem, In Computers & Operations Research, Volume 38, Issue 8, 2011, Pages 1153-1163, ISSN 0305-0548, https://doi.org/10.1016/j.cor.2010.11.009.

Utilizaremos el siguiente modelo

$\min \sum_{(u,v)\in E} c_{uv}^+ x_{uv} - \theta$


s.t.

$\theta \leq \sum_{(u,v)\in E} c_{uv}^- y_{uv} + \sum_{(u,v)\in E} y_{uv} (c_{uv}^+ - c_{uv}^-) x_{uv}\qquad \forall Y\in A$

$\sum_{u\in U} x_{uv} = 1, \qquad v\in V$;

$\sum_{v\in V} x_{uv} = 1, \qquad u\in U$;

$x_{uv}\in \{0,1\}, \qquad u\in U, v\in V$.

Nótese que la restricción que calcula $\theta$ hace uso de diversas propiedades del problema (sabemos que el escenario que maximiza la diferencia entre dos soluciones es un escenario extremo de características concretas y que una de esas soluciones define la peor alternativa y por tanto el worst-case regret de una solución $X$.

En realidad es más conveniente implementar como:

$\min \theta$


s.t.

$\theta \geq \sum_{(u,v)\in E} c_{uv}^s x_{uv} - c_{uv}^s x_{uv}^{\ast}\forall s\in S$

y específicamente definir $s$ para una solución $x$ como su caso extremo. Esto es, $c_{uv}^s= c_{uv}^+$ si $x_{uv}=1$ y $c_{uv}^s= c_{uv}^-$ en el resto de los casos.

In [2]:
function doInstanciaAleatoria(n::Int64)
    cplus = rand(1:1000000, n*n)
    cminus=map(x -> rand(1:x), cplus)
    return reshape(cminus,n,n),reshape(cplus,n,n)
end

doInstanciaAleatoria (generic function with 1 method)

In [3]:
srand(0)
cminus,cplus=doInstanciaAleatoria(15)

(
[46212 227032 … 594252 384268; 282754 404683 … 80822 529592; … ; 256077 100788 … 695986 130348; 84569 130215 … 42096 11657],

[50241 229958 … 734015 665675; 498707 524136 … 104588 898982; … ; 375631 140089 … 737225 519511; 152652 153509 … 55125 35154])

In [4]:
function solveAP(c,n::Int64,solver)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n,1:n],Bin)
    @objective(submodel,Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n) )
    
    @constraint(submodel, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(submodel, destination[j = 1:n], sum(x[1:n,j]) == 1)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

solveAP (generic function with 1 method)

In [5]:
function solveRAP(cminus, cplus, n::Int64, solver=CplexSolver())
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    
    
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido

    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

solveRAP (generic function with 2 methods)

In [ ]:
solveRAP(cminus,cplus,15)

## Compliquemos ligeramente el ejemplo anterior

Tenemos dos puntos que podríamos mejorar:

(1) No estamos generando cortes para las soluciones fraccionales

(2) Vamos a usar la heuristic callback para generar posibles soluciones. Debido a la particularidad de la lazy cut, ésta será la encargada de insertar los cortes

La forma de implementarla va a ser un tanto "rara" y se debe a particularidades de este problema. 

Adicionalmente añadimos una "information callback"

In [6]:
#Instead of x += y, use append!(x,y) to modify x in place. If y is a single variable, you may also use push!(x, coef, y) in place of x += coef*y.                


function solveRAPwithHeuristic(cminus, cplus, n::Int64, solver=CplexSolver(CPX_PARAM_SCRIND=0))
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    pending = JuMP.GenericAffExpr[]
    function heuristic(m)
        EPSILON=0.00001
        x_val = getvalue(x)
        xTest=zeros(Int64,n)
        yUsed=zeros(Int64,n)
        bestVal=0
        for i in 1:n
            val=(-1)
            bestVal=0
            for j in 1:n
                if yUsed[j]==0 && val<x_val[i,j]
                    val=x_val[i,j]
                    bestVal=j
                end
            end
            xTest[i]=bestVal
            yUsed[bestVal]=1
        end
        
        cost=deepcopy(cminus)        
        for i in 1:n
            x_val[i,:]=0
            x_val[i,xTest[i]]=1
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        evaluacionCost = - optimalCost
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
                if x_val[i,j]>EPSILON
                    evaluacionCost += cost[i,j]
                end
            end
        end
        #println("\nIn: ",MathProgBase.cbgetobj(m)," vs ",evaluacionCost)
        if(MathProgBase.cbgetobj(m)>evaluacionCost)
            push!(pending,rhs)
            setsolutionvalue(m,θ,evaluacionCost)
            for i in 1:n
                for j in 1:n
                    if x_val[i,j]>EPSILON
                        setsolutionvalue(m,x[i,j],1)
                    else
                        setsolutionvalue(m,x[i,j],0)
                    end
                end
            end
            addsolution(m)
        end
    end
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    function cutGenerator(m)
        while !isempty(pending)
            rhs = pop!(pending)
            @usercut(m,θ>=rhs)
        end
    end
    function infocallback(m)
        println("exploredNodes: ",MathProgBase.cbgetexplorednodes(m)," obj: ",MathProgBase.cbgetobj(m), " bestBound: ",MathProgBase.cbgetbestbound(m))        
    end
    #:MIPNode if at node in branch-and-cut tree, 
    #:MIPSol at an integer-feasible solution,
    #:Intermediate otherwise.
    addinfocallback(m, infocallback, when = :Intermediate)
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    addcutcallback(m,cutGenerator)
    addheuristiccallback(m,heuristic)
    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

solveRAPwithHeuristic (generic function with 2 methods)

In [7]:
solveRAPwithHeuristic(cminus,cplus,15)

exploredNodes: 0 obj: 1.0e75 bestBound: -2.8257e6
exploredNodes: 0 obj: 1.0e75 bestBound: -2.8257e6
exploredNodes: 0 obj: 1.0e75 bestBound: 0.0
exploredNodes: 0 obj: 1.0e75 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBoun

exploredNodes: 59 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 59 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 60 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 60 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 61 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 62 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 63 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 63 obj: 1.200952e6 bestBound: 699985.6502837229
exploredNodes: 64 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 64 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 65 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 65 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 66 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 66 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 67 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 67 obj: 1.200952e6 bestBound: 710611.940

exploredNodes: 117 obj: 1.520952e6 bestBound: 737423.838441673
exploredNodes: 117 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 118 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 118 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 119 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 119 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 120 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 120 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 121 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 121 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 122 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 122 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 123 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 123 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 124 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 125 obj: 1.200952e6 bestBound: 737423.83

exploredNodes: 189 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 189 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 190 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 190 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 191 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 191 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 192 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 192 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 193 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 193 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 194 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 194 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 195 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 195 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 196 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 196 obj: 1.200952e6 bestB

exploredNodes: 262 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 262 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 263 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 263 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 264 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 264 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 265 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 265 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 266 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 266 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 267 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 267 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 268 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 268 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 269 obj: 1.200952e6 bestBound: 796078.7842053397
exploredNodes: 269 obj: 1.200952e6 bestB

...

exploredNodes: 2625 obj: 1.200952e6 bestBound: 1.1345576414572108e6
exploredNodes: 2626 obj: 1.200952e6 bestBound: 1.1350142199413225e6
exploredNodes: 2627 obj: 1.200952e6 bestBound: 1.13516401548585e6
exploredNodes: 2628 obj: 1.200952e6 bestBound: 1.1353269867502484e6
exploredNodes: 2628 obj: 1.200952e6 bestBound: 1.1353269867502484e6
exploredNodes: 2629 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2630 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2630 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2631 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2632 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2632 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2633 obj: 1.200952e6 bestBound: 1.135593858133639e6
exploredNodes: 2634 obj: 1.200952e6 bestBound: 1.1356705447579958e6
exploredNodes: 2635 obj: 1.200952e6 bestBound: 1.1357170601577586e6
exploredNodes: 2636 obj: 1.200952e6 bestBound: 1.13573651

exploredNodes: 2737 obj: 1.200952e6 bestBound: 1.1474486919251957e6
exploredNodes: 2738 obj: 1.200952e6 bestBound: 1.1474486919251957e6
exploredNodes: 2739 obj: 1.200952e6 bestBound: 1.1474486919251957e6
exploredNodes: 2739 obj: 1.200952e6 bestBound: 1.1474486919251957e6
exploredNodes: 2740 obj: 1.200952e6 bestBound: 1.1475521638569932e6
exploredNodes: 2741 obj: 1.200952e6 bestBound: 1.1475521638569932e6
exploredNodes: 2742 obj: 1.200952e6 bestBound: 1.1475994614341157e6
exploredNodes: 2742 obj: 1.200952e6 bestBound: 1.1475994614341157e6
exploredNodes: 2743 obj: 1.200952e6 bestBound: 1.147764479714643e6
exploredNodes: 2744 obj: 1.200952e6 bestBound: 1.147764479714643e6
exploredNodes: 2745 obj: 1.200952e6 bestBound: 1.147764479714643e6
exploredNodes: 2746 obj: 1.200952e6 bestBound: 1.1478946813583712e6
exploredNodes: 2747 obj: 1.200952e6 bestBound: 1.148124906052608e6
exploredNodes: 2748 obj: 1.200952e6 bestBound: 1.1484481874902025e6
exploredNodes: 2748 obj: 1.200952e6 bestBound: 1.148

exploredNodes: 2840 obj: 1.310952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2840 obj: 1.200952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2841 obj: 1.200952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2842 obj: 1.200952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2843 obj: 1.200952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2843 obj: 1.200952e6 bestBound: 1.1592572110843246e6
exploredNodes: 2844 obj: 1.200952e6 bestBound: 1.1595423161376407e6
exploredNodes: 2845 obj: 1.200952e6 bestBound: 1.1595799856322813e6
exploredNodes: 2846 obj: 1.200952e6 bestBound: 1.1596054682669186e6
exploredNodes: 2847 obj: 1.200952e6 bestBound: 1.1596506637350512e6
exploredNodes: 2848 obj: 1.200952e6 bestBound: 1.1597369828903545e6
exploredNodes: 2849 obj: 1.200952e6 bestBound: 1.159912296760134e6
exploredNodes: 2850 obj: 1.200952e6 bestBound: 1.1599198945942807e6
exploredNodes: 2851 obj: 1.200952e6 bestBound: 1.1599577606518276e6
exploredNodes: 2852 obj: 1.200952e6 bestBound: 1.

exploredNodes: 2985 obj: 1.200952e6 bestBound: 1.176654258745094e6
exploredNodes: 2986 obj: 1.200952e6 bestBound: 1.17670589062742e6
exploredNodes: 2987 obj: 1.200952e6 bestBound: 1.1770239476948364e6
exploredNodes: 2987 obj: 1.200952e6 bestBound: 1.1770239476948364e6
exploredNodes: 2988 obj: 1.200952e6 bestBound: 1.1770685156626685e6
exploredNodes: 2989 obj: 1.200952e6 bestBound: 1.1770685156626685e6
exploredNodes: 2990 obj: 1.200952e6 bestBound: 1.1770685156626685e6
exploredNodes: 2991 obj: 1.200952e6 bestBound: 1.1771896575019006e6
exploredNodes: 2992 obj: 1.200952e6 bestBound: 1.1774850494784696e6
exploredNodes: 2993 obj: 1.200952e6 bestBound: 1.1775082029610288e6
exploredNodes: 2994 obj: 1.200952e6 bestBound: 1.1776368300982735e6
exploredNodes: 2995 obj: 1.200952e6 bestBound: 1.1778915960696891e6
exploredNodes: 2996 obj: 1.200952e6 bestBound: 1.1779057621665895e6
exploredNodes: 2997 obj: 1.200952e6 bestBound: 1.177928939596426e6
exploredNodes: 2998 obj: 1.200952e6 bestBound: 1.177

exploredNodes: 3142 obj: 1.200952e6 bestBound: 1.1974952752518144e6
exploredNodes: 3143 obj: 1.200952e6 bestBound: 1.197564844089188e6
exploredNodes: 3144 obj: 1.200952e6 bestBound: 1.197749030819546e6
exploredNodes: 3145 obj: 1.200952e6 bestBound: 1.197852394412153e6
exploredNodes: 3146 obj: 1.200952e6 bestBound: 1.1979241160642875e6
exploredNodes: 3147 obj: 1.200952e6 bestBound: 1.1980967673428732e6
exploredNodes: 3148 obj: 1.200952e6 bestBound: 1.198885257156295e6
exploredNodes: 3149 obj: 1.200952e6 bestBound: 1.1991076374444107e6
exploredNodes: 3150 obj: 1.200952e6 bestBound: 1.1999069881316768e6
exploredNodes: 3151 obj: 1.200952e6 bestBound: 1.200129601387127e6
exploredNodes: 3152 obj: 1.200952e6 bestBound: 1.200182522225273e6
exploredNodes: 3153 obj: 1.200952e6 bestBound: 1.2005622648362645e6
exploredNodes: 3154 obj: 1.200952e6 bestBound: 1.2006149460934652e6
exploredNodes: 3155 obj: 1.200952e6 bestBound: 1.2006208481828328e6
exploredNodes: 3156 obj: 1.200952e6 bestBound: 1.20071